# InfluxDB Client for Funding Rate
##### Anish Shourie



In [12]:
import pandas as pd
import numpy as np
from influxdb import DataFrameClient
from datetime import datetime
from datetime import timedelta

In [16]:
# run this cell to create a new database in influxDB

symbol = 'XBT'
host = 'localhost'
port = 8086

def initialize_db(symbol, host, port):
    
    current_date = datetime.today().date() - timedelta(days = 1)

    temp_date = pd.Timestamp("1900-1-1").date()
    data = pd.DataFrame()
    c = 0

    while temp_date <= current_date:

        url = "https://www.bitmex.com/api/v1/funding?_format=csv&count=500&start="+str(c)+"&symbol=" + symbol + "&reverse=false"
        df_temp = pd.read_csv(url, parse_dates=['timestamp'])
        data = data.append(df_temp)
        c += 500
        temp_date = df_temp.loc[len(df_temp)-1, 'timestamp'].date()
        
    data['time'] = pd.to_datetime(data['timestamp'])
    data.insert(0, 'date', data['time'].dt.date)
    data.insert(1, 'funding_time', data['time'].dt.time)
    data = data.drop('time', axis=1)
    data = data.set_index('timestamp')
    
    dbname = symbol+'_funding_rate'
    protocol = 'line'
    client = DataFrameClient(host=host, port=port, database=dbname)

    print("Create database: " + dbname)
    client.create_database(dbname)

    print("Write DataFrame to " + dbname)
    client.write_points(data, dbname, protocol=protocol)

    return True

initialize_db(symbol, host, port)

Create database: XBT_funding_rate
Write DataFrame to XBT_funding_rate


True

In [17]:
# Run this cell to update the already created Database with new funding rate data

def update_db(client, symbol, n):
    
    dbname = symbol+'_funding_rate'
    
    current_date = pd.Timestamp.now().date()

    temp_date = pd.Timestamp("1900-1-1").date()
    data = pd.DataFrame()

    while temp_date <= current_date:

        url = "https://www.bitmex.com/api/v1/funding?_format=csv&count=500&start="+str(n)+"&symbol=" + symbol + "&reverse=false"
        try:
            df_temp = pd.read_csv(url, parse_dates=['timestamp'])
        except:
            print("Database already updated")
            return False
        data = data.append(df_temp)
        n += 500
        if len(df_temp) < 500:
            break
        temp_date = df_temp.loc[len(df_temp)-1, 'timestamp'].date()
        
    data['time'] = pd.to_datetime(data['timestamp'])
    data.insert(0, 'date', data['time'].dt.date)
    data.insert(1, 'funding_time', data['time'].dt.time)
    data = data.drop('time', axis=1)
    data = data.set_index('timestamp')
    
    protocol = 'line'

    print("Updated " + dbname)
    client.write_points(data, dbname, protocol=protocol)
    
    return True

symbol = 'XBT'
host = 'localhost'
port = 8086

client = DataFrameClient(host=host, port=port, database= symbol+'_funding_rate')
q = client.query('SELECT * FROM XBT_funding_rate')
data = q['XBT_funding_rate']
n = len(data)

update_db(client, symbol, n)

Database already updated


False

In [18]:
# Run this cell to query data into a dataframe called "data"

symbol = 'XBT'
host = 'localhost'
port = 8086

client = DataFrameClient(host=host, port=port, database= symbol+'_funding_rate')
q = client.query('SELECT * FROM XBT_funding_rate')
data = q['XBT_funding_rate']

In [19]:
# Running this cell filters for dates that only have 3 funding periods in that day

data3 = pd.DataFrame()

for index, row in data.iterrows():
    date = row['date']
    temp = data[data['date'] == date]
    if len(temp) == 3:
        data3 = data3.append(temp)
data3 = data3.drop_duplicates(subset= ['date', 'funding_time'])

In [20]:
data3.to_csv("kobe.csv")

In [10]:
data3['isAbove'] = data3['fundingRate'] >= 0.0001
data3['isBelow'] = data3['fundingRate'] <= -0.0001
data3['above_count'] = np.nan
data3['below_count'] = np.nan

In [11]:
above_count = 0
below_count = 0

for index, row in data3.iterrows():
    if row['isAbove']:
        above_count+=1
    elif row['isAbove'] is False:
        above_count=0
    data3.loc[index, 'above_count'] = above_count
    
    if row['isBelow']:
        below_count+=1
    elif row['isBelow'] is False:
        below_count=0
        
    data3.loc[index, 'below_count'] = below_count

data3

,date,fundingInterval,fundingRate,fundingRateDaily,funding_time,symbol,isAbove,isBelow,above_count,below_count
2016-06-05 04:00:00+00:00,2016-06-05,2000-01-01T08:00:00.000Z,0.000242,0.000727,04:00:00,XBTUSD,True,False,1.0,0.0
2016-06-05 12:00:00+00:00,2016-06-05,2000-01-01T08:00:00.000Z,0.000237,0.000712,12:00:00,XBTUSD,True,False,2.0,0.0
2016-06-05 20:00:00+00:00,2016-06-05,2000-01-01T08:00:00.000Z,0.000234,0.000703,20:00:00,XBTUSD,True,False,3.0,0.0
2016-06-06 04:00:00+00:00,2016-06-06,2000-01-01T08:00:00.000Z,0.000229,0.000687,04:00:00,XBTUSD,True,False,4.0,0.0
2016-06-06 12:00:00+00:00,2016-06-06,2000-01-01T08:00:00.000Z,0.000240,0.000721,12:00:00,XBTUSD,True,False,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-11-02 12:00:00+00:00,2020-11-02,2000-01-01T08:00:00.000Z,-0.000015,-0.000045,12:00:00,XBTUSD,False,False,0.0,0.0
2020-11-02 20:00:00+00:00,2020-11-02,2000-01-01T08:00:00.000Z,-0.000140,-0.000420,20:00:00,XBTUSD,False,True,0.0,1.0
2020-11-03 04:00:00+00:00,2020-11-03,2000-01-01T08:00:00.000Z,-0.000197,-0.000591,04:00:00,XBTUSD,False,True,0.0,2.0
2020-11-03 12:00:00+00:00,2020-11-03,2000-01-01T08:00:00.000Z,0.000077,0.000231,12:00:00,XBTUSD,False,False,0.0,0.0


1. What is the average time_period it spends above 1 and below? 
2. What is the cumulative funding rate? Sum of the funding rates when cumulative days above or below 1?
3. What is the likelihood of funding rate deviating from 1 again in the next 1 period?

In [30]:
# average number of intervals above 0.0001 and below -0.0001

tot_list_a = []
temp_list_a = []
for index, row in data3.iterrows():
    if row['above_count'] >= 1:
        temp_list_a.append(index)
    else:
        if len(temp_list_a) > 0:
            tot_list_a.append(temp_list_a)
        temp_list_a = []
        
tot_list_b = []
temp_list_b = []
for index, row in data3.iterrows():
    if row['below_count'] >= 1:
        temp_list_b.append(index)
    else:
        if len(temp_list_b) > 0:
            tot_list_b.append(temp_list_b)
        temp_list_b = []
        
above_intervals = []
below_intervals = []
for i in tot_list_a:
    above_intervals.append(len(i))
print("The average number of intervals above 0.0001: " + str(sum(above_intervals)/len(above_intervals)))
for j in tot_list_b:
    below_intervals.append(len(j))
print("The average number of intervals below -0.0001: " + str(sum(below_intervals)/len(below_intervals)))

The average number of intervals above 0.0001: 6.390829694323144
The average number of intervals below -0.0001: 3.198511166253102


In [46]:
# average cumulative funding rate when periods above 0.0001 and below -0.0001 (includes streak of 1 day)

tot_list_a = []
temp_a = 0
for index, row in data3.iterrows():
    if row['above_count'] >= 1:
        temp_a += row['fundingRate']
    else:
        if temp_a != 0:
            tot_list_a.append(temp_a)
        temp_a = 0

tot_list_b = []
temp_b = 0
for index, row in data3.iterrows():
    if row['below_count'] >= 1:
        temp_b += row['fundingRate']
    else:
        if temp_b != 0:
            tot_list_b.append(temp_b)
        temp_b = 0
        
print("Average cumulative funding rate when consecutive periods above 0.0001: " + str(sum(tot_list_a)/len(tot_list_a)))
print("Average cumulative funding rate when consecutive periods below -0.0001: " + str(sum(tot_list_b)/len(tot_list_b)))

Average cumulative funding rate when consecutive periods above 0.0001: 0.0038396360989810677
Average cumulative funding rate when consecutive periods below -0.0001: -0.0028614292803970207


In [47]:
one_period = []
two_period = []
three_period = []
four_period = []

for i in range (0, len(data3) - 5):
    if (data3.iloc[i,6] or data3.iloc[i,7]) and data3.iloc[i+1,6] == False:
        if data3.iloc[i+2,6] or data3.iloc[i+2,7]:
            one_period.append(True)
            two_period.append(True)
            three_period.append(True)
            four_period.append(True)
        elif data3.iloc[i+3,6] or data3.iloc[i+3,7]:
            one_period.append(False)
            two_period.append(True)
            three_period.append(True)
            four_period.append(True)
        elif data3.iloc[i+4,6] or data3.iloc[i+4,7]:
            one_period.append(False)
            two_period.append(False)
            three_period.append(True)
            four_period.append(True)
        elif data3.iloc[i+5,6] or data3.iloc[i+5,7]:
            one_period.append(False)
            two_period.append(False)
            three_period.append(False)
            four_period.append(True)
        else:
            one_period.append(False)
            two_period.append(False)
            three_period.append(False)
            four_period.append(False)

In [53]:
print("Probability deviates again after 1 period: " + str(sum(one_period)/len(one_period)))
print("Probability deviates again after 2 periods: " + str(sum(two_period)/len(two_period)))
print("Probability deviates again after 3 periods: " + str(sum(three_period)/len(three_period)))
print("Probability deviates again after 4 periods: " + str(sum(four_period)/len(four_period)))

Probability deviates again after 1 period: 0.8223383409536251
Probability deviates again after 2 periods: 0.9359895493141738
Probability deviates again after 3 periods: 0.9732201175702155
Probability deviates again after 4 periods: 0.9895493141737427
